# Automatic Image Colorization


## 1. Image Colorization Using CNN 

### Colorful Image Colorization 
Richard Zhang, Phillip Isola, Alexei A. Efros

[<a href="https://arxiv.org/pdf/1603.08511.pdf">Paper</a>][<a href="https://github.com/richzhang/colorization">Code</a>][<a href="http://videolectures.net/eccv2016_zhang_image_colorization/">Video</a>]


Initiallly a RGB image is broken up in LAB Color Space [<a href="https://en.wikipedia.org/wiki/Lab_color_space">more on lab</a>] which is somewhat similar to the $YC_bC_r$ channel we are taught in class .Here the L channel is the Lightness or simply the grayscale version of the image . The ab Channel is what that determines the colorfulness of the image ,a and b for the color opponents green–red and blue–yellow.
### This is how the channels of lab colorspace looks
![lab image](imgs/lab.png)

### The Naive CNN 
![cnn](imgs/cnn.png)
![naive cnn](imgs/naive_cnn.png)

This is how the Naive CNN looks on the VGG Architeture . We simply put the L channel in CNN and put the ab channel as target here $L\in R^{HxW}$ and $ab \in R^{HxWx2}$ we Train on L and want to predict the ab and then merge both of them to get the colorized image . We can frame this as a per pixel regression problem Where our object is to minimize the L2 Objective function or simply the euclidean distance . If our ground truth from ab is Y and the predicted ab is Y'. We say our objective is to minimize . 
$$L_2=\frac{1}{2}\sum_{h,w}||Y-Y'||^2$$ 

![ab space](imgs/ab_space.png)

But our Objective Function is kinda ill formed . <b>Why ?</b>

As They mention in the paper , 

<i>"However, this loss is not robust to the inherent ambiguity and multimodal
nature  of  the  colorization  problem.  If  an  object  can  take  on  a  set  of  distinct
ab values,  the  optimal  solution  to  the  Euclidean  loss  will  be  the  mean  of  the
set. In color prediction, this averaging effect favors grayish, desaturated results.
Additionally, if the set of plausible colorizations is non-convex, the solution will
in fact be out of the set, giving implausible results."</i>

<b>PS : I don't understand how this objective function acctually results in mean and discourages extreme prediction .</b>

Instead we should treat this problem as a classification one 

![loss 2](imgs/loss_2.png)


Instead we treat the problem as a multimodal classification one , here we quantize the ab space in $Q=313$ bins of size 10x10 . For a given input $X$ we try to find a mapping $Z'=G(X)$ where $Z \in [0,1]^{HxWxQ}$ .

Further to improve the coloring we use the probability distribution of colors in ab space for natural images . so our new loss stands at. This additional term encourages our model to given high weightages to naturally occuring color . more on this can be found on <b>2.2 Class Rebalancing </b> section of the paper.
![loss 3](imgs/loss_3.png)
 
What is <b>$v(Z_{h,w})$</b> ? Nothing but the emperical probablity distribution for that specific ab bin . What emperical Probality distribution looks like ? 

![proba](imgs/emperical_proba.png)

Finally we map the probality distribution $Z'$ to color values $Y'$ using a function $Y'=H(Z')$ , more on this can be found on the <b>2.3 Class Probabilities to Point Estimates</b> section of the paper.

So the final architure is somewhat like this 

![final CNN](imgs/new_cnn.png)